In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')


In [5]:
df.head(4)

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0

In [6]:
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [7]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [10]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [14]:
X

array([[ 0.99720391, -0.57873591, -0.57380915, ...,  0.64609167,
         0.97024255,  0.02188649],
       [-1.00280393, -0.57873591,  1.74273971, ..., -1.54776799,
         0.97024255,  0.21653375],
       [ 0.99720391, -0.57873591, -0.57380915, ...,  0.64609167,
        -1.03067011,  0.2406869 ],
       ...,
       [ 0.99720391, -0.57873591, -0.57380915, ..., -1.54776799,
         0.97024255, -1.00864308],
       [-1.00280393,  1.72790383, -0.57380915, ...,  0.64609167,
        -1.03067011, -0.12523071],
       [ 0.99720391, -0.57873591, -0.57380915, ...,  0.64609167,
        -1.03067011, -1.07636976]])

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20, random_state=0)

In [16]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 5,activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 4,activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid'))


In [17]:
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [18]:
ann.fit(X_train,y_train,batch_size = 32,epochs = 15,validation_data=(X_test,y_test))

Epoch 1/15
250/250 [==============================] - 4s 6ms/step - loss: 0.6087 - accuracy: 0.7360 - val_loss: 0.5118 - val_accuracy: 0.7975
Epoch 2/15
250/250 [==============================] - 2s 8ms/step - loss: 0.4861 - accuracy: 0.7960 - val_loss: 0.4643 - val_accuracy: 0.7975
Epoch 3/15
250/250 [==============================] - 2s 8ms/step - loss: 0.4434 - accuracy: 0.7960 - val_loss: 0.4255 - val_accuracy: 0.7975
Epoch 4/15
250/250 [==============================] - 1s 5ms/step - loss: 0.4121 - accuracy: 0.8005 - val_loss: 0.3933 - val_accuracy: 0.8195
Epoch 5/15
250/250 [==============================] - 1s 3ms/step - loss: 0.3827 - accuracy: 0.8338 - val_loss: 0.3676 - val_accuracy: 0.8500
Epoch 6/15
250/250 [==============================] - 1s 3ms/step - loss: 0.3668 - accuracy: 0.8450 - val_loss: 0.3570 - val_accuracy: 0.8575
Epoch 7/15
250/250 [==============================] - 1s 3ms/step - loss: 0.3591 - accuracy: 0.8491 - val_loss: 0.3531 - val_accuracy: 0.8590
Epoch 

In [19]:
y_pred = ann.predict(X_test)

63/63 [==============================] - 1s 7ms/step


In [20]:
y_pred = (y_pred > 0.5)

In [21]:
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),axis = 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix

ac = accuracy_score(y_test,y_pred)
ac

0.862

In [23]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[1509,   86],
       [ 190,  215]])